# Exercise 02: Camera Model

<div>
<img src="data/Problem_Formulation.png" width="800"/>
</div>


As illustrated in the picture above, the goal is to code a reprojector. It would allow to transform a pixel on a first image plane into a pixel on a second image plane, given the distance of the corresponding 3D point with respect to the first camera, and the camera parameters.
The reprojector will simply execute the following steps:
1) Unproject the pixel from image plane 1 using the distance value, to obtain a 3D point in camera 1 coordinate.
2) Convert this 3D point to camera 2 coordinate.
3) Project it to image plane 2.

It will be first done in case of a perfect pinhole camera model, then distortion will be taken into account through the FOV model that was described in the fourth exercise sheet.

In [7]:
from exercise_code.camera import *

from tests import test_pinhole, test_fov, test_reprojection, test_relative_pose
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Camera coordinates conversion
As stated previously, a conversion from camera 1 to camera 2 coordinates is needed. However, the camera poses that are provided allows to move from camera to world coordinate, and are given as $3 \times 4$ matrices of the form $[ R_i | T_i ]$. Thus a function that would allow to obtain the relative pose between the two cameras is convenient. Such a relative pose would also be a $3 \times 4$ matrix of the form $[ R | T]$, and transforms indeed from camera 1 to camera 2 coordinate.

Go to file ```/exercise_code/camera.py``` and implement the function ```compute_relative_pose```.

Run the following test:

In [8]:
test_relative_pose()

INFO:root:Congratulations: You passed the test case for the relative pose.
INFO:root:Score: 100/100


100

### Pinhole camera model
In this section, we assume that both cameras are ideal, and follow the pinhole camera model described in the lecture. A class ```Pinhole``` has been defined to simulate such a camera, and contains two methods. The first one ```project``` takes as input a point in camera coordinate, and output a pixel in the image plane. And the second one ```unproject``` takes as input a pixel and a distance, and output the corresponding 3D point in camera coordinate (whose projection is the given pixel, and whose distance from the camera origin is the given distance).

 Go to file ```/exercise_code/camera.py``` and implement those functions of the class ```Pinhole```.

Check, whether you passed the test:

In [9]:
test_pinhole()

INFO:root:Congratulations: You passed the test case for the pinhole projection.
INFO:root:Congratulations: You passed the test case for the pinhole unprojection.
INFO:root:All tests of PinholeTest passed.
Tests passed: 2/2
INFO:root:Score: 100/100


100

### FOV camera model
Now we assume that the cameras exhibit some radial distortion according to the FOV model, please refer to the second part of the fourth exercise sheet for the details.
Similar to the pinhole camera case, a class ```Fov``` have been defined to simulate such a camera, and have the exact same methods as the class ```Pinhole``` implemented previously.

Go to file ```/exercise_code/camera.py``` and implement those functions of the class ```Fov```.

Check, whether you passed the test:

In [10]:
test_fov()

INFO:root:Congratulations: You passed the test case for the FOV projection.
INFO:root:Congratulations: You passed the test case for the FOV unprojection.
INFO:root:All tests of FovTest passed.
Tests passed: 2/2
INFO:root:Score: 100/100


100

### Final code and test

In [14]:
import numpy as np

data = np.load("data/data.npz")

# Creating cameras
first_cam = Pinhole(640,480,600,600,320,240)
second_cam = Pinhole(640,480,600,600,320,240)

# Load data
pixels = data["pixels"]
distances = data["distances"]
pose_1 = data["pose_1"]
pose_2 = data["pose_2"]
relative_pose = compute_relative_pose(pose_1,pose_2)
npix = distances.size

reprojected_pixels = data["reprojected_pixels_1"]

for i in range(npix):
    # Take a pixel of image 1, unproject it, transform to cam 2, project to image 2
    pt = first_cam.unproject(pixels[i],distances[i])

    pt_cam2 = relative_pose @ np.append(pt,1.0) #Transform to cam 2 coordinates
    pix2 = second_cam.project(pt_cam2) #Project to image 2

    reprojection_error = np.linalg.norm(pix2 - reprojected_pixels[i, :])
    print("reprojected pixel:({0},{1}), reprojection error: {2}".format(pix2[0],pix2[1],reprojection_error))


reprojected pixel:(561.2318770072492,235.04299362681456), reprojection error: 4.5089409502570425e-12
reprojected pixel:(664.0137189555777,95.37099118856386), reprojection error: 5.213524708774418e-12
reprojected pixel:(469.9828696976405,319.54769594054136), reprojection error: 3.945195362453122e-12


In [12]:
test_reprojection()

INFO:root:Congratulations: You passed the test case for the pinhole reprojection.
INFO:root:Congratulations: You passed the test case for the FOV reprojection.
INFO:root:All tests of ReprojectionTest passed.
Tests passed: 2/2
INFO:root:Score: 100/100


100

### Submit

In [13]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/exercise02')

relevant folders: ['exercise_code']
notebooks files: ['camera_model.ipynb']
Adding folder exercise_code
Adding notebook camera_model.ipynb
Zipping successful! Zip is stored under: /Users/andreas/Projects/TUM/cv2mvg/output/exercise02.zip
